In [ ]:
from google.colab import drive
import os


drive.mount('/content/drive', force_remount=True)
folder_path = '/content/drive/My Drive/MSc Computer Science/Master Thesis/'

for f in os.listdir(folder_path):
    print(f)

Mounted at /content/drive
literature-review-mt.gsheet
testing.ipynb
Archive
Annotations
Thesis Notes.gdoc
Copy of topic_modeing.ipynb
bertb-embeddings
bertl-embeddings
trial_data
baselines.ipynb
code
topic_modeing.ipynb
annotations.ipynb
train_data.xlsx
iaa.ipynb
trial_base_embeddings
Supervision: Detecting Idiosyncratic Sense Shift.gdoc
ap.ipynb
finetuning.ipynb


In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
from datasets import Dataset
import pandas as pd

train_df = pd.read_excel('/content/drive/My Drive/MSc Computer Science/Master Thesis/train_data.xlsx')
train_dataset = Dataset.from_pandas(train_df[['text']])

In [ ]:
train_dataset

Dataset({
    features: ['text'],
    num_rows: 273934
})

In [ ]:
from transformers import AutoTokenizer

model_name = "NbAiLab/nb-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/273934 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments

output_dir = "/content/drive/MyDrive/nb-bert-finetuned"

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    learning_rate=2e-5,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
)

In [ ]:
import os

checkpoint_path = None
if os.path.isdir(output_dir):
    checkpoints = [ckpt for ckpt in os.listdir(output_dir) if ckpt.startswith("checkpoint")]
    if checkpoints:
        latest_ckpt = sorted(checkpoints, key=lambda x: int(x.split("-")[-1]))[-1]
        checkpoint_path = os.path.join(output_dir, latest_ckpt)
        print(f"Resuming from checkpoint: {checkpoint_path}")


Resuming from checkpoint: /content/drive/MyDrive/nb-bert-finetuned/checkpoint-37500


In [ ]:
print(checkpoint_path)

/content/drive/MyDrive/nb-bert-finetuned/checkpoint-37500


7d8a5ddd0d2fc5f1167f1481502991bafecc2daf

In [ ]:
if checkpoint_path:
    trainer.train(resume_from_checkpoint=checkpoint_path)
else:
    trainer.train()


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anitavn (anitavn-university-of-copenhagen) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
38000,0.566500
38500,0.562900
39000,0.568700
39500,0.562500
40000,0.560300
40500,0.564300
41000,0.561300
41500,0.564100
42000,0.556900
42500,0.560200


In [ ]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/nb-bert-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/nb-bert-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/nb-bert-finetuned/vocab.txt',
 '/content/drive/MyDrive/nb-bert-finetuned/added_tokens.json',
 '/content/drive/MyDrive/nb-bert-finetuned/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModel.from_pretrained(output_dir)

model.eval()

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/nb-bert-finetuned and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals